In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r
import pandas_datareader as pdr
from dotenv import load_dotenv
import os
load_dotenv()
token = os.getenv("ALPHA")
from time import sleep

In [3]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [4]:
market.connect()
start = market.retrieve_max_date("prices")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2022-10-13 2022-10-14


In [5]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("prices",data)
        except:
            print(ticker,"tiingo")
        try:
            test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                    , start=start
                                                    , end=end)
            panda_reader_data = test.read()
            panda_reader_data["ticker"] = ticker
            panda_reader_data.reset_index(inplace=True)
            market.store("pdr_prices",panda_reader_data)
        except:
            print(ticker,"pdr")
        sleep(1)
    except Exception as e:
        print(str(e))
market.disconnect()

 22%|███████████████▍                                                     | 111/497 [04:59<17:02,  2.65s/it]

market prices documents must be a non-empty list
CTXS pdr


 32%|█████████████████████▊                                               | 157/497 [06:59<15:36,  2.76s/it]

market prices documents must be a non-empty list
DRE pdr


 66%|█████████████████████████████████████████████▊                       | 330/497 [18:45<11:16,  4.05s/it]

market prices documents must be a non-empty list


100%|█████████████████████████████████████████████████████████████████████| 497/497 [29:51<00:00,  3.60s/it]


In [6]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████| 497/497 [00:18<00:00, 26.95it/s]


In [7]:
market.connect()
try:
    try:
        test = pdr.yahoo.daily.YahooDailyReader(symbols="BTC-USD", start=start, end=end)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = "BTC"
        panda_reader_data.reset_index(inplace=True)
        market.store("crypto",panda_reader_data)
    except:
        print("btc","pdr")
except Exception as e:
    print(str(e))
market.disconnect()

In [8]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
152,-0.394122,0.053154,0.053154,0.076178,0.104388,DPZ,Domino's,Consumer Discretionary,Restaurants
475,-0.506514,0.046025,0.046025,0.039933,0.072041,WBD,Warner Bros. Discovery,Communication Services,Broadcasting
264,-0.271002,0.014572,0.014572,0.048306,0.071841,KEY,KeyCorp,Financials,Regional Banks
192,-0.219029,0.046299,0.046299,0.041540,0.066108,FITB,Fifth Third Bank,Financials,Regional Banks
237,-0.100289,0.031180,0.031180,0.049887,0.066002,HBAN,Huntington National Bank,Financials,Regional Banks
67,-0.370278,-0.011681,-0.011681,0.037559,0.065769,BLK,BlackRock,Financials,Asset Management & Custody Banks
392,-0.055884,0.017544,0.017544,0.030602,0.065306,RF,Regions Financial Corporation,Financials,Regional Banks
66,0.105677,0.021339,0.021339,0.059502,0.064138,BIIB,Biogen,Health Care,Biotechnology
100,-0.496680,0.017704,0.017704,0.024003,0.064085,CHTR,Charter Communications,Communication Services,Cable & Satellite
4,-0.295422,0.011483,0.011483,0.044731,0.063064,ABMD,Abiomed,Health Care,Health Care Equipment


In [9]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Energy,0.477568,0.096549,0.096549,0.041751,0.038752
Consumer Staples,-0.042772,0.022565,0.022565,0.028928,0.017276
Financials,-0.156637,0.015543,0.015543,0.023230,0.039356
Consumer Discretionary,-0.302653,0.013669,0.013669,0.012521,0.008850
Industrials,-0.164172,0.009792,0.009792,0.010470,0.018907
Health Care,-0.166476,0.006089,0.006089,0.021003,0.024970
Materials,-0.163346,0.003116,0.003116,0.010653,0.027193
Communication Services,-0.303448,-0.007465,-0.007465,0.007696,0.025036
Information Technology,-0.301702,-0.030871,-0.030871,-0.001450,0.024268
